In [1]:
import sys
sys.path.insert(0, 'C:\\Users\\Sen\\Documents\\GitHub\\neurobay\\')

%reload_ext autoreload
%autoreload 2
%aimport neurobay

import numpy as np
import pandas as pd
import csv
from importlib import reload
import neurobay as nb
import patsy

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


### Native example from ComBat script

In [2]:
# Load data
pheno = pd.read_table('C:\\Users\\Sen\\Documents\\GitHub\\neurobay\\examples\\bladder-pheno.txt', index_col=0)
dat = pd.read_table('C:\\Users\\Sen\\Documents\\GitHub\\neurobay\\examples\\bladder-expr.txt', index_col=0)

In [3]:
# Make the model matrix describing metadata on the samples which could be causing batch effects. 
mod = patsy.dmatrix("~ age + cancer", pheno, return_type="dataframe")
mod.head()

,Intercept,cancer[T.Cancer],cancer[T.Normal],age
cel,,,,
GSM71019.CEL,1.0,0.0,1.0,1.0
GSM71020.CEL,1.0,0.0,1.0,2.0
GSM71021.CEL,1.0,0.0,1.0,3.0
GSM71022.CEL,1.0,0.0,1.0,4.0
GSM71023.CEL,1.0,0.0,1.0,5.0


In [4]:
# The batch to be corrected
pheno['batch'].head()

cel
GSM71019.CEL    3
GSM71020.CEL    2
GSM71021.CEL    2
GSM71022.CEL    3
GSM71023.CEL    3
Name: batch, dtype: int64

In [5]:
help(nb.combat.combat)

Help on function combat in module neurobay.combat:

combat(data, batch, estimators=None, model=None, numerical_covariates=None)
    Correct for batch effects in a dataset
    
    Args:
        data : pandas.DataFrame, 
        A (n_features, n_samples) dataframe of the expression or methylation data to batch correct
        ----------------------------------
        batch : pandas.Series, 
        A column corresponding to the batches in the data, with index same as the columns that appear in ``data``
        ----------------------------------
        estimators: (optional) list of 2 np.ndarray, 
        the trained batch effect estimators from another data.[additive_effect, multiplicative_effect], default=None
        ----------------------------------
        model : (optional) patsy.design_info.DesignMatrix, 
        A model matrix describing metadata on the samples which could be causing batch effects. 
        If not provided, then will attempt to coarsely correct just from the i

In [6]:
# Do the ComBat batch correction
ebat,estimators = nb.combat.combat(dat, pheno['batch'], None, mod, "age")

found 5 batches
found 1 numerical covariates...
	age
found 2 categorical variables:	cancer[T.Cancer], cancer[T.Normal]
Standardizing Data across genes.
Fitting L/S model and finding priors
No valid imported estimators found
Calculating estimators
Finding parametric adjustments


Adjusting data
Complete


### Compare original data and batch corrected data

In [7]:
# Original data descriptions
dat.describe()

,GSM71019.CEL,GSM71020.CEL,GSM71021.CEL,GSM71022.CEL,GSM71023.CEL,GSM71024.CEL,GSM71025.CEL,GSM71026.CEL,GSM71028.CEL,GSM71029.CEL,...,GSM71068.CEL,GSM71069.CEL,GSM71070.CEL,GSM71071.CEL,GSM71072.CEL,GSM71073.CEL,GSM71074.CEL,GSM71075.CEL,GSM71076.CEL,GSM71077.CEL
count,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,...,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000
mean,6.130612,6.105730,6.148067,6.110766,6.136627,6.148246,6.096199,6.086093,6.127455,6.148102,...,6.134862,6.142437,6.125205,6.109861,6.137484,6.087813,6.095000,6.129819,6.119351,6.109156
std,1.643439,1.430372,1.445430,1.512157,1.634152,1.786840,1.484436,1.458656,1.886841,1.579223,...,1.847278,1.704540,1.638163,1.619317,1.635150,1.552515,1.426240,1.670713,1.597142,1.343078
min,2.889960,2.878797,2.990483,2.951296,2.939028,2.961916,2.807166,2.955576,2.841143,2.974328,...,2.815274,2.883244,2.910467,2.965756,3.043127,2.963154,2.856256,2.818975,2.936577,2.900967
25%,4.947762,5.112792,5.148865,5.060637,4.948368,4.792351,5.073876,5.080051,4.628732,5.007417,...,4.688308,4.897190,4.946743,4.963685,4.959545,4.988161,5.100280,4.925170,4.978456,5.175101
50%,6.058526,6.090444,6.130098,6.074515,6.046650,5.963585,6.066165,6.072252,5.915767,6.028924,...,5.934116,6.017412,6.028068,6.008674,6.041059,6.015328,6.098255,6.019468,6.040812,6.138471
75%,7.092359,6.973610,7.030684,7.027194,7.099428,7.253738,6.987259,6.972869,7.374765,7.059012,...,7.350271,7.154183,7.078267,7.044369,7.081554,6.992879,6.990987,7.087919,7.060615,6.989433
max,13.762109,13.699280,13.682598,13.612873,13.661680,13.615698,13.623534,13.529756,13.386938,13.705754,...,13.818925,13.695921,13.805094,13.684099,13.851343,13.646309,13.824596,13.663660,13.710441,13.574468


In [8]:
# Corrected data descriptions
ebat.describe()

,GSM71019.CEL,GSM71020.CEL,GSM71021.CEL,GSM71022.CEL,GSM71023.CEL,GSM71024.CEL,GSM71025.CEL,GSM71026.CEL,GSM71028.CEL,GSM71029.CEL,...,GSM71068.CEL,GSM71069.CEL,GSM71070.CEL,GSM71071.CEL,GSM71072.CEL,GSM71073.CEL,GSM71074.CEL,GSM71075.CEL,GSM71076.CEL,GSM71077.CEL
count,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,...,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000
mean,6.122445,6.105152,6.147860,6.102311,6.131690,6.143469,6.097185,6.082200,6.128101,6.143239,...,6.138661,6.129386,6.110952,6.113515,6.139208,6.093075,6.101122,6.111341,6.104767,6.118060
std,1.502768,1.452413,1.458710,1.429971,1.485539,1.592694,1.498261,1.464077,1.842366,1.617452,...,1.808446,1.634398,1.583805,1.604301,1.621930,1.542265,1.433334,1.600321,1.563766,1.413069
min,2.911021,2.984865,3.002526,2.976692,2.927385,2.985128,2.931548,2.916550,2.844134,3.004780,...,2.887706,2.864312,2.825849,2.932408,2.978011,2.878718,2.865101,2.392276,2.708414,2.463299
25%,5.073775,5.100317,5.134958,5.117787,5.089983,4.971632,5.054527,5.071125,4.688236,4.967053,...,4.744044,4.957186,4.983741,4.984115,4.977801,5.005656,5.102058,4.962636,4.993931,5.128943
50%,6.116391,6.087219,6.141680,6.077193,6.112056,6.073940,6.071474,6.069431,5.946946,6.019347,...,5.958413,6.033262,6.031638,6.021127,6.050111,6.018692,6.097330,6.029722,6.036695,6.121503
75%,7.031469,6.981989,7.035156,6.966505,7.027099,7.122038,6.985714,6.965086,7.311978,7.088528,...,7.287364,7.079974,7.037582,7.031217,7.073025,6.995704,6.999299,7.041225,7.027545,7.034003
max,13.733378,14.013743,13.574380,14.301185,13.600553,13.609132,13.599704,13.513358,13.324993,13.674508,...,13.744998,13.661354,13.770305,13.641671,13.807459,13.714747,13.768995,13.678633,13.713254,14.205990


### Looking at the content of the estimators

In [9]:
# Additive batch effect collector
gamma_star = estimators[0]
gamma_star.shape
gamma_star

array([[-0.39739443, -0.30641535,  0.11490446, ..., -0.25596522,
        -0.16996147, -0.56775535],
       [ 0.15303961, -0.27453668,  0.55014048, ...,  1.33991682,
         0.71777493,  1.238842  ],
       [ 2.1134704 ,  0.07644276, -0.62860032, ..., -1.20676715,
        -0.94945938, -0.20652865],
       [ 0.79241312,  0.33021717, -0.91702513, ..., -0.74074774,
        -0.04430869, -0.71521132],
       [-0.4770065 ,  0.35383098, -0.22642061, ..., -0.55287476,
        -0.3136371 , -0.53267585]])

In [10]:
# Multiplicative batch effect collector
delta_star = estimators[1]
delta_star.shape
delta_star

array([[1.01989195, 1.40243318, 0.86652482, ..., 0.94598271, 1.07606166,
        0.8310675 ],
       [0.64617444, 1.22538829, 1.4792203 , ..., 0.96603005, 0.93019755,
        1.15893326],
       [0.91990302, 0.57726679, 0.62844092, ..., 0.6436439 , 0.79867371,
        1.56635385],
       [0.90942697, 0.57550964, 0.65793243, ..., 0.86028397, 1.16978731,
        1.01006681],
       [1.51621635, 0.927691  , 0.96689037, ..., 1.34695019, 1.15946793,
        0.96108622]])

### Batch correct with pre-trained estimators

In [11]:
### Use pre-trained estimators to do ComBat
ebat2,estimators = nb.combat.combat(dat, pheno['batch'], estimators, mod, "age")

found 5 batches
found 1 numerical covariates...
	age
found 2 categorical variables:	cancer[T.Cancer], cancer[T.Normal]
Standardizing Data across genes.
Fitting L/S model and finding priors
Imported estimators found


Adjusting data
Complete


### The 2 results are identical

In [12]:
# Corrected data descriptions
ebat.describe()

,GSM71019.CEL,GSM71020.CEL,GSM71021.CEL,GSM71022.CEL,GSM71023.CEL,GSM71024.CEL,GSM71025.CEL,GSM71026.CEL,GSM71028.CEL,GSM71029.CEL,...,GSM71068.CEL,GSM71069.CEL,GSM71070.CEL,GSM71071.CEL,GSM71072.CEL,GSM71073.CEL,GSM71074.CEL,GSM71075.CEL,GSM71076.CEL,GSM71077.CEL
count,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,...,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000
mean,6.122445,6.105152,6.147860,6.102311,6.131690,6.143469,6.097185,6.082200,6.128101,6.143239,...,6.138661,6.129386,6.110952,6.113515,6.139208,6.093075,6.101122,6.111341,6.104767,6.118060
std,1.502768,1.452413,1.458710,1.429971,1.485539,1.592694,1.498261,1.464077,1.842366,1.617452,...,1.808446,1.634398,1.583805,1.604301,1.621930,1.542265,1.433334,1.600321,1.563766,1.413069
min,2.911021,2.984865,3.002526,2.976692,2.927385,2.985128,2.931548,2.916550,2.844134,3.004780,...,2.887706,2.864312,2.825849,2.932408,2.978011,2.878718,2.865101,2.392276,2.708414,2.463299
25%,5.073775,5.100317,5.134958,5.117787,5.089983,4.971632,5.054527,5.071125,4.688236,4.967053,...,4.744044,4.957186,4.983741,4.984115,4.977801,5.005656,5.102058,4.962636,4.993931,5.128943
50%,6.116391,6.087219,6.141680,6.077193,6.112056,6.073940,6.071474,6.069431,5.946946,6.019347,...,5.958413,6.033262,6.031638,6.021127,6.050111,6.018692,6.097330,6.029722,6.036695,6.121503
75%,7.031469,6.981989,7.035156,6.966505,7.027099,7.122038,6.985714,6.965086,7.311978,7.088528,...,7.287364,7.079974,7.037582,7.031217,7.073025,6.995704,6.999299,7.041225,7.027545,7.034003
max,13.733378,14.013743,13.574380,14.301185,13.600553,13.609132,13.599704,13.513358,13.324993,13.674508,...,13.744998,13.661354,13.770305,13.641671,13.807459,13.714747,13.768995,13.678633,13.713254,14.205990


In [13]:
# Corrected using pre-trained estimators
ebat2.describe()

,GSM71019.CEL,GSM71020.CEL,GSM71021.CEL,GSM71022.CEL,GSM71023.CEL,GSM71024.CEL,GSM71025.CEL,GSM71026.CEL,GSM71028.CEL,GSM71029.CEL,...,GSM71068.CEL,GSM71069.CEL,GSM71070.CEL,GSM71071.CEL,GSM71072.CEL,GSM71073.CEL,GSM71074.CEL,GSM71075.CEL,GSM71076.CEL,GSM71077.CEL
count,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,...,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000,22283.000000
mean,6.122445,6.105152,6.147860,6.102311,6.131690,6.143469,6.097185,6.082200,6.128101,6.143239,...,6.138661,6.129386,6.110952,6.113515,6.139208,6.093075,6.101122,6.111341,6.104767,6.118060
std,1.502768,1.452413,1.458710,1.429971,1.485539,1.592694,1.498261,1.464077,1.842366,1.617452,...,1.808446,1.634398,1.583805,1.604301,1.621930,1.542265,1.433334,1.600321,1.563766,1.413069
min,2.911021,2.984865,3.002526,2.976692,2.927385,2.985128,2.931548,2.916550,2.844134,3.004780,...,2.887706,2.864312,2.825849,2.932408,2.978011,2.878718,2.865101,2.392276,2.708414,2.463299
25%,5.073775,5.100317,5.134958,5.117787,5.089983,4.971632,5.054527,5.071125,4.688236,4.967053,...,4.744044,4.957186,4.983741,4.984115,4.977801,5.005656,5.102058,4.962636,4.993931,5.128943
50%,6.116391,6.087219,6.141680,6.077193,6.112056,6.073940,6.071474,6.069431,5.946946,6.019347,...,5.958413,6.033262,6.031638,6.021127,6.050111,6.018692,6.097330,6.029722,6.036695,6.121503
75%,7.031469,6.981989,7.035156,6.966505,7.027099,7.122038,6.985714,6.965086,7.311978,7.088528,...,7.287364,7.079974,7.037582,7.031217,7.073025,6.995704,6.999299,7.041225,7.027545,7.034003
max,13.733378,14.013743,13.574380,14.301185,13.600553,13.609132,13.599704,13.513358,13.324993,13.674508,...,13.744998,13.661354,13.770305,13.641671,13.807459,13.714747,13.768995,13.678633,13.713254,14.205990
